In [ ]:
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/command_line_interface.html
# ref： https://cooltools.readthedocs.io/en/latest/notebooks/dots.html
import os 
print(os.getcwd())
os.chdir("/data1/sunwenju/projects_on_940xa/2024041703_mESC_HiC_use_pairtools_cooler_pipeline/Nora_WT")
print(os.getcwd())

In [ ]:
! ls -1 *.mcool

In [ ]:
nora_wt_rep12_mcool = "Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_wt_rep1_mcool = "Nora_mESC_WT_HiC_rep1.nodups.pairs.1k.zoomify.mcool"
nora_wt_rep2_mcool = "Nora_mESC_WT_HiC_rep2.nodups.pairs.1k.zoomify.mcool"

In [ ]:
import cooler
# to print which resolutions are stored in the mcool, use list_coolers
cooler.fileops.list_coolers(nora_wt_rep12_mcool)

In [ ]:
import pandas as pd
mm10_view_df = pd.read_csv('../mm10.chrom.view.tsv', sep='\t', header=0)
mm10_view_df.head()

# nora_wt_rep12_mcool

## r20k dots and loops calling

In [ ]:
import pandas as pd
import cooler
import cooltools
resolution=20_000
mm10_view_df = pd.read_csv("../mm10.chrom.view.tsv", sep="\t", header=0)
nora_wt_rep12_mcool = "Nora_mESC_WT_HiC.rep12.merged.1k.zoomify.mcool"
nora_wt_rep12_mcool_r20k = cooler.Cooler(f'{nora_wt_rep12_mcool}::resolutions/{resolution}')
nora_wt_rep12_mcool_r20k_expected_cis = cooltools.expected_cis(
    clr=nora_wt_rep12_mcool_r20k,
    view_df = mm10_view_df,
    intra_only=True,
    smooth=True,
    aggregate_smoothed=True,
    nproc=64
)
nora_wt_rep12_mcool_r20k_expected_cis.to_csv("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.expected_cis.tsv", sep="\t", index=False, header=True, na_rep="NaN")
# 第6步 进行 insulation & TAD 分析 时已经计算过了，此时直接读入对应的 expected_cis 数据文件即可
# nora_wt_rep12_mcool_r20k_expected_cis = pd.read_csv("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.expected_cis.tsv", sep="\t", header=0)
print(nora_wt_rep12_mcool_r20k_expected_cis.shape)
nora_wt_rep12_mcool_r20k_expected_cis.head()


In [ ]:
nora_wt_rep12_mcool_r20k_dots_df = cooltools.dots(
    clr=nora_wt_rep12_mcool_r20k,
    expected=nora_wt_rep12_mcool_r20k_expected_cis,
    view_df=mm10_view_df,
    # how far from the main diagonal to call dots:
    max_loci_separation=10_000_000,
    max_nans_tolerated=5,
    nproc=128,
)
nora_wt_rep12_mcool_r20k_dots_df.to_csv("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.dots_info.tsv", sep="\t", index=False, header=True)


In [ ]:
print(nora_wt_rep12_mcool_r20k_dots_df.shape)
nora_wt_rep12_mcool_r20k_dots_df.head()

## loops view plot

In [ ]:
import bioframe
# Visualization imports:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
# helper functions for plotting
bp_formatter = EngFormatter('b')
def format_ticks(ax, x=True, y=True, rotate=True):
    """format ticks with genomic coordinates as human readable"""
    if y:
        ax.yaxis.set_major_formatter(bp_formatter)
    if x:
        ax.xaxis.set_major_formatter(bp_formatter)
        ax.xaxis.tick_bottom()
    if rotate:
        ax.tick_params(axis='x',rotation=45)

# create a functions that would return a series of rectangles around called dots
# in a specific region, and exposing importnat plotting parameters
def rectangles_around_dots(dots_df, resolution, region, loc="upper", lw=1, ec="cyan", fc="none"):
    """
    yield a series of rectangles around called dots in a given region
    """
    # select dots from the region:
    df_reg = bioframe.select(
        bioframe.select(dots_df, region, cols=("chrom1","start1","end1")),
        region,
        cols=("chrom2","start2","end2"),
    )
    rectangle_kwargs = dict(lw=lw, ec=ec, fc=fc)
    # draw rectangular "boxes" around pixels called as dots in the "region":
    for s1, s2, e1, e2 in df_reg[["start1", "start2", "end1", "end2"]].itertuples(index=False):
        width1 = e1 - s1
        width2 = e2 - s2
        if loc == "upper":
            yield patches.Rectangle((s2+resolution/5, s1+resolution/5), width2, width1, **rectangle_kwargs)
        elif loc == "lower":
            yield patches.Rectangle((s1+resolution/5, s2+resolution/5), width1, width2, **rectangle_kwargs)
        else:
            raise ValueError("loc has to be uppper or lower")


### Fig2A loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr1"
start = 16_000_000
end = 18_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Fig2A ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Fig2A.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Fig2A.fall.heatmap.with_loops.png")


### Fig2B loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr9"
start = 13_800_000
end = 15_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Fig2B ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Fig2B.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Fig2B.fall.heatmap.with_loops.png")


### Fig3E loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr1"
start = 9_800_000
end = 14_600_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Fig3E ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Fig3E.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Fig3E.fall.heatmap.with_loops.png")


### FigS3J loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr14"
start = 109_400_000
end = 111_900_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k FigS3J ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.FigS3J.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.FigS3J.fall.heatmap.with_loops.png")


### Malat1 loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr19"
start = 5_000_000
end = 6_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1500, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Malat1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Malat1.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Malat1.fall.heatmap.with_loops.png")


### Myc loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr15"
start = 61_000_000
end = 63_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Myc ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Myc.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Myc.fall.heatmap.with_loops.png")


### Nanog loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr6"
start = 121_500_000
end = 123_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1200, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Nanog ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Nanog.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Nanog.fall.heatmap.with_loops.png")


### Pax6 loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr2"
start = 104_500_000
end = 106_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Pax6 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Pax6.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Pax6.fall.heatmap.with_loops.png")


### Pou5f1 loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr17"
start = 34_500_000
end = 36_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1600, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Pou5f1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Pou5f1.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Pou5f1.fall.heatmap.with_loops.png")


### Sox1 loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr8"
start = 11_000_000
end = 13_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1100, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Sox1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Sox1.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Sox1.fall.heatmap.with_loops.png")


### Sox2 loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr3"
start = 33_500_000
end = 35_500_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=1000, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Sox2 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Sox2.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Sox2.fall.heatmap.with_loops.png")


### Zic1 loops view

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cooltools.lib.plotting
from cooltools.lib.numutils import fill_diag
from cooltools.lib.numutils import adaptive_coarsegrain, interp_nan
# define a region to look into as an example
chromsome = "chr9"
start = 89_000_000
end = 93_000_000
plot_region = (chromsome, start, end)
ax_limit_extents = (start, end, end, start)
# Smoothing & Interpolation
nora_wt_rep12_mcool_r20k_adcg = adaptive_coarsegrain(nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(plot_region),
                                nora_wt_rep12_mcool_r20k.matrix(balance=False).fetch(plot_region),
                                cutoff=3, max_levels=9)
nora_wt_rep12_mcool_r20k_adcg_interp = interp_nan(nora_wt_rep12_mcool_r20k_adcg)

# # # compute heatmap for the region
# # region_matrix = nora_wt_rep12_mcool_r20k.matrix(balance=True).fetch(region)
# for diag in [-2, -1, 0, 1, 2]:
#     nora_wt_rep12_mcool_r20k_adcg_interp = fill_diag(nora_wt_rep12_mcool_r20k_adcg_interp, np.nan, i=diag)

fig = plt.figure(figsize=(6, 5.4))
fig.subplots_adjust(left=0.16, right=0.88, bottom=0.12, top=0.96, wspace=0.08)
# fig.suptitle("xxx_plot")
# ax1
ax1 = fig.add_subplot(1, 1, 1)
# im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg*100000, vmin=0, vmax=1300, cmap='fall', extent=ax_limit_extents)
im = ax1.matshow(nora_wt_rep12_mcool_r20k_adcg_interp*100000, vmin=0, vmax=800, cmap='fall', extent=ax_limit_extents)
plt.colorbar(im, ax=ax1 ,fraction=0.046, pad=0.04, label='(balance=True)');
ax1.set_title(f'Nora_WT_r20k Zic1 ({chromsome}:{start:,}-{end:,})', y=1.01)
ax1.set_xlabel('position(Mb)')
ax1.set_ylabel('position(Mb)')
format_ticks(ax1)

# draw rectangular "boxes" around pixels called as dots in the "region":
for box in rectangles_around_dots(nora_wt_rep12_mcool_r20k_dots_df, resolution, plot_region, lw=1.5, loc="upper"):
    ax1.add_patch(box)

plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Zic1.fall.heatmap.with_loops.pdf")
plt.savefig("Nora_mESC_WT_HiC.rep12.merged.mcool_r20k.view.Zic1.fall.heatmap.with_loops.png")
